In [1]:
import pandas as pd
import numpy as np

In [2]:
def two_dice_sum():
    d1 = np.random.choice(range(6))+1 
    d2 = np.random.choice(range(6))+1
    return d1+d2,d1,d2

def select_house(value):
    return min(max(value,3),11)


In [3]:
award_his = pd.DataFrame()
for trail in range(10000):
    #2 df, dice history & in time position
    dice_his = pd.DataFrame()
    
    #house2-11, extra move if dice again, current position, is last diced
    int_status = [{'house':3,'extra move':3,'colour':'blue'},
                {'house':4,'extra move':3,'colour':'blue'},
                {'house':5,'extra move':2,'colour':'orange'},
                {'house':6,'extra move':1,'colour':'red'},
                {'house':7,'extra move':0,'colour':'black'},
                {'house':8,'extra move':1,'colour':'red'},
                {'house':9,'extra move':2,'colour':'orange'},
                {'house':10,'extra move':3,'colour':'blue'},
                {'house':11,'extra move':3,'colour':'blue'},

                ]
    position = pd.DataFrame(int_status)
    position[['current position','last diced']] = [0,0]


    while position['current position'].max() <15:
        dice_sum,d1,d2 = two_dice_sum()
        selected_house = select_house(dice_sum)
        selected_house_inf = position[position['house'] == selected_house].iloc[0]
        move = 1 if selected_house_inf['last diced'] == 0 else 1 + selected_house_inf['extra move']

        record_dict = [{'selected_house':selected_house,'move':move,'dice_sum':dice_sum,'d1':d1,'d2':d2}]
        dice_his = pd.concat([dice_his,pd.DataFrame(record_dict)])

        position.loc[position['house']==selected_house,'current position'] = position.loc[position['house']==selected_house,'current position'] + move
        position['last diced'] = 0
        position.loc[position['house']==selected_house,'last diced'] = 1

    position['rank'] = position['current position'].rank(ascending=False,method='min')
    award_dict = {}

    for i in range(3,12):
        award_dict['house {} rank'.format(i)] = position[position['house']==i]['rank'].iloc[0]
    award_dict['rank1_2 diff'] = position['current position'].max() - position['current position'].nlargest(2).min()
    award_dict['rank1_3 diff'] = position['current position'].max() - position['current position'].nlargest(3).min()
    award_dict['min_position'] = position['current position'].min()
    award_dict['rank1_colour'] = position[position['rank'] == 1]['colour'].iloc[0]
    award_dict['house moved <=3'] = position[position['current position'] <=3].count()['house']
    award_dict['diced times'] = dice_his.count().iloc[0]
    award_dict['boosted times'] = dice_his[dice_his['move'] > 1].count().iloc[0]
    
    award_his = pd.concat([award_his,pd.DataFrame(award_dict,index=[trail])])

award_his['house 7 rank>=5'] = award_his['house 7 rank'] >=5
award_his['2house posit <=3'] = award_his['house moved <=3'] >=2
award_his['min_position>=6'] = award_his['min_position'] >= 6
award_his['rank1_2 diff <=1'] = award_his['rank1_2 diff'] <=1
award_his['rank1_3 diff <=3'] = award_his['rank1_3 diff'] <=3

display(award_his)

,house 3 rank,house 4 rank,house 5 rank,house 6 rank,house 7 rank,house 8 rank,house 9 rank,house 10 rank,house 11 rank,rank1_2 diff,...,min_position,rank1_colour,house moved <=3,diced times,boosted times,house 7 rank>=5,2house posit <=3,min_position>=6,rank1_2 diff <=1,rank1_3 diff <=3
0,7.0,7.0,9.0,2.0,1.0,5.0,4.0,5.0,2.0,4,...,3,black,1,67,4,False,False,False,False,False
1,5.0,6.0,2.0,6.0,6.0,9.0,1.0,4.0,3.0,8,...,1,orange,5,27,8,True,True,False,False,False
2,7.0,3.0,5.0,4.0,1.0,5.0,2.0,9.0,7.0,1,...,3,black,1,71,6,False,False,False,True,True
3,2.0,5.0,1.0,5.0,5.0,5.0,9.0,4.0,2.0,9,...,1,orange,1,41,5,True,False,False,False,False
4,3.0,7.0,1.0,2.0,3.0,5.0,7.0,5.0,9.0,3,...,3,orange,1,74,6,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,2.0,9.0,3.0,1.0,3.0,5.0,6.0,8.0,7.0,3,...,3,red,1,66,7,False,False,False,False,False
9996,9.0,3.0,3.0,3.0,2.0,6.0,1.0,6.0,8.0,4,...,4,orange,0,71,7,False,False,False,False,False
9997,7.0,7.0,2.0,4.0,3.0,6.0,1.0,7.0,4.0,2,...,5,orange,0,67,5,False,False,False,False,False
9998,5.0,6.0,3.0,3.0,6.0,1.0,2.0,6.0,6.0,1,...,3,red,4,52,6,True,True,False,True,False


In [4]:
review_rank = award_his[['house 3 rank',
                         'house 4 rank',
                         'house 5 rank',
                         'house 6 rank',
                         'house 7 rank',
                         'house 8 rank',
                         'house 9 rank',
                         'house 10 rank',
                         'house 11 rank',
                         ]]
display('rank 1 probability',(review_rank == 1).mean())
display('rank 1-2 probability',(review_rank <=2).mean())
display('rank 1-3 probability',(review_rank <=3).mean())
display('average info',award_his.mean())

'rank 1 probability'

house 3 rank     0.0478
house 4 rank     0.0519
house 5 rank     0.1133
house 6 rank     0.1797
house 7 rank     0.2048
house 8 rank     0.1807
house 9 rank     0.1207
house 10 rank    0.0509
house 11 rank    0.0502
dtype: float64

'rank 1-2 probability'

house 3 rank     0.1256
house 4 rank     0.1326
house 5 rank     0.2529
house 6 rank     0.3875
house 7 rank     0.4553
house 8 rank     0.3873
house 9 rank     0.2671
house 10 rank    0.1286
house 11 rank    0.1298
dtype: float64

'rank 1-3 probability'

house 3 rank     0.2064
house 4 rank     0.2123
house 5 rank     0.3751
house 6 rank     0.5464
house 7 rank     0.6441
house 8 rank     0.5445
house 9 rank     0.3905
house 10 rank    0.2043
house 11 rank    0.2029
dtype: float64

C:\Users\XSEA0081\AppData\Local\Temp\ipykernel_8468\2126585400.py:14: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  display('average info',award_his.mean())


'average info'

house 3 rank         5.8327
house 4 rank         5.7865
house 5 rank         4.5323
house 6 rank         3.5785
house 7 rank         3.1154
house 8 rank         3.5789
house 9 rank         4.4475
house 10 rank        5.8526
house 11 rank        5.8462
rank1_2 diff         3.5160
rank1_3 diff         5.1903
min_position         2.8049
house moved <=3      1.4680
diced times         61.5263
boosted times        5.4980
house 7 rank>=5      0.2142
2house posit <=3     0.3996
min_position>=6      0.0652
rank1_2 diff <=1     0.2121
rank1_3 diff <=3     0.2808
dtype: float64

In [5]:
award_his[['rank1_colour','diced times']].groupby('rank1_colour').count().sort_values('diced times')


,diced times
rank1_colour,
blue,2008
black,2048
orange,2340
red,3604
